In [53]:
from schrodinger.application.desmond.packages import traj_util, traj, topo
#from schrodinger.application.desmond.packages import traj_util, traj, topo
import numpy as np
import pandas as pd

import os
cms_path = os.path.expanduser('~/Dropbox/KcsA/kcsa_5000ns_1218/kcsa_deactivated_5us_centered-out.cms')
traj_path = os.path.expanduser('~/Dropbox/KcsA/kcsa_5000ns_1218/kcsa_deactivated_5us_centered_trj')

# Load structure and trajectory (paths to your .cms and .trj files)
_, cms_model = topo.read_cms(cms_path)
trajectory = traj.read_traj(traj_path)

# Select backbone atoms: carbonyl carbon (C), oxygen (O), and amide nitrogen (N)
c_aids = cms_model.select_atom("chain.name A and res.num 21 and atom.ptype C")
o_aids = cms_model.select_atom("chain.name A and res.num 76 and atom.ptype O")
n_aids = cms_model.select_atom("chain.name A and res.num 76 and atom.ptype N")

#print(c_aids)

# Map atom IDs to global IDs for trajectory processing
gid_C_list = topo.aids2gids(cms_model, c_aids)
gid_O_list = topo.aids2gids(cms_model, o_aids)
gid_N_list = topo.aids2gids(cms_model, n_aids)

# Ensure the lists are of equal length (one-to-one correspondence)
assert len(gid_C_list) == len(gid_O_list) == len(gid_N_list), "Mismatch in atom selections."

# Initialize lists to accumulate orientation vectors over time
sigma_33_list, sigma_22_list, sigma_11_list = [], [], []

# Loop through trajectory frames
for frame in trajectory:
    # Process each residue (assume one-to-one correspondence of C, O, N per residue)
    for gid_C, gid_O, gid_N in zip(gid_C_list, gid_O_list, gid_N_list):
        # Extract atomic positions
        C = frame.pos(gid_C)
        #print(C)
        O = frame.pos(gid_O)
        #print(O)
        N = frame.pos(gid_N)
        #print(N)

        # Build local molecular frame
        vec_CO = O - C
        sigma_22 = vec_CO / np.linalg.norm(vec_CO)  # Along CO bond

        # Normal vector to peptide plane (C', O, N)
        vec_CN = N - C
        sigma_33 = np.cross(vec_CO, vec_CN)  # Perpendicular to peptide plane
        sigma_33 /= np.linalg.norm(sigma_33)

        # Orthogonal axis
        sigma_11 = np.cross(sigma_33, sigma_22)  # Orthogonal to sigma_33 and sigma_22
        sigma_11 /= np.linalg.norm(sigma_11)

        # Append to orientation lists
        sigma_33_list.append(sigma_33)
        sigma_22_list.append(sigma_22)
        sigma_11_list.append(sigma_11)

# Post-processing: Save or analyze the accumulated orientation data
sigma_33_array = np.array(sigma_33_list)
sigma_22_array = np.array(sigma_22_list)
sigma_11_array = np.array(sigma_11_list)

#print(sigma_33_array.shape)



frame_times = np.array([fr.time for fr in trajectory])  # Extract time for each frame in the trajectory

# Create a DataFrame
df = pd.DataFrame({
    'Frame': np.arange(len(frame_times)),  # Frame indices
    'Time (ps)': frame_times,             # Trajectory time
    'Sigma_33_X': sigma_33_array[:,0],   # X-component of Sigma_33
    'Sigma_33_Y': sigma_33_array[:,1],   # Y-component of Sigma_33
    'Sigma_33_Z': sigma_33_array[:,2],   # Z-component of Sigma_33
    'Sigma_22_X': sigma_22_array[:,0],   # X-component of Sigma_22
    'Sigma_22_Y': sigma_22_array[:,1],   # Y-component of Sigma_22
    'Sigma_22_Z': sigma_22_array[:,2],   # Z-component of Sigma_22
    'Sigma_11_X': sigma_11_array[:,0],   # X-component of Sigma_11
    'Sigma_11_Y': sigma_11_array[:,1],   # Y-component of Sigma_11
    'Sigma_11_Z': sigma_11_array[:,2],   # Z-component of Sigma_11
})

# Save the DataFrame to an Excel file
output_file = "orientation_vectors.xlsx"
df.to_excel(output_file, index=False)

print(f"Orientation data saved to {output_file}")

<ipython-input-53-170f02b6059b>:90: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.9' currently installed).
  df.to_excel(output_file, index=False)


Orientation data saved to orientation_vectors.xlsx


In [66]:
# from the local frame, we can calculate the rotation matrix from local frame to lab frame
sigma11= 240   #
sigma22= 190   # y
sigma33= 95

csa_principle=[[240,0, 0],[0,190,0],[0,0,95]]
#print(sigma_33_array.shape)
print(sigma_33_array[0])
print(sigma_22_array[0])
print(sigma_11_array[0])

r_t = np.stack((sigma_33_array,sigma_22_array,sigma_11_array),axis=-1)
r_t = np.transpose(r_t, axes=(0, 2, 1))
print(r_t[0,:])  # to verify the rotation matrix is right

# now calculat sigma(t)

sigma_t = np.einsum('nij,jk,nkl->nil', r_t, csa_principle, r_t.transpose(0, 2, 1))

[-0.20971954 -0.91157365 -0.35362566]
[ 0.7928684   0.05309229 -0.6070757 ]
[ 0.57216907 -0.40769428  0.7116235 ]
[[-0.20971954 -0.91157365 -0.35362566]
 [ 0.7928684   0.05309229 -0.6070757 ]
 [ 0.57216907 -0.40769428  0.7116235 ]]


In [2]:
import numpy as np

# Example: use u_z_list from above to build A(t)
u_z_arr = np.array(u_z_list)            # shape (Nframes,3)
cos2 = u_z_arr[:,2]**2                 # z-component squared
A = (3*cos2 - 1) * (Delta_sigma) / 2.0  # CSA interaction sequence

# Autocorrelation (unnormalized, one-sided)
N = len(A)
A_mean = np.mean(A)
acf = np.array([np.mean((A[:N-k]-A_mean)*(A[k:]-A_mean)) for k in range(N)])


2


In [ ]:

# Time step (ps) between frames; convert to s if needed for SI units
dt = fr.time  # e.g. 0.002 ps per frame or fr.time attribute
# Compute FFT of acf (use one-sided convention)
J_full = np.real(np.fft.rfft(acf)) * dt
freq = np.fft.rfftfreq(N, dt)  # cycles per ps
omega = 2*np.pi * freq         # rad per ps

gamma13 = 2*np.pi * 10.705e6  # rad/T/s (gyromag ratio 13C)
omega0 = gamma13 * B0         # rad/s
# Convert anisotropy (ppm) to rad/s: sigma_rad = Delta_sigma_ppm * omega0 * 1e-6
sigma_rad = Delta_sigma_ppm * omega0 * 1e-6
R1 = (2.0/15.0) * (omega0**2) * (sigma_rad**2) * J_omega0
T1 = 1.0 / R1
